In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("d:/TrainAndValid.csv")

In [ ]:
df.info()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(df["saledate"][:1000], df["SalePrice"][:1000])

In [ ]:
df.SalePrice.plot.hist()

In [ ]:
df = pd.read_csv("d:/TrainAndValid.csv", low_memory = False, parse_dates = ['saledate'])

In [ ]:
fig, ax = plt.subplots()
ax.scatter(df["saledate"][:1000], df["SalePrice"][:1000])

In [ ]:
df.head()

In [ ]:
df.sort_values(by=['saledate'], inplace=True, ascending=True)
df.saledate.head(20)

In [ ]:
df_tmp = df.copy()

In [ ]:
df_tmp["saleYear"] = df_tmp.saledate.dt.year
df_tmp["saleMonth"] = df_tmp.saledate.dt.month
df_tmp["saleDay"] = df_tmp.saledate.dt.day
df_tmp["saleDayOfWeek"] = df_tmp.saledate.dt.dayofweek
df_tmp["saleDayOfYear"] = df_tmp.saledate.dt.dayofyear

In [ ]:
df_tmp.drop("saledate", axis=1, inplace=True)

In [ ]:
df_tmp.info()

In [ ]:
pd.api.types.is_string_dtype(df_tmp["UsageBand"])

In [ ]:
for label, content in df_tmp.items():
    if pd.api.types.is_string_dtype(content):
        print(label)

In [ ]:
for label, content in df_tmp.items():
    if pd.api.types.is_string_dtype(content):
        df_tmp[label] = content.astype("category").cat.as_ordered()

In [ ]:
df_tmp.info()

In [ ]:
df_tmp.state.cat.categories

In [ ]:
df_tmp.state.cat.codes

In [ ]:
### Saving the processed data
df_tmp.to_csv("d:/df-temp-bulldozer.csv")

In [ ]:
df_tmp = pd.read_csv("d:/df-temp-bulldozer.csv", low_memory = False)

In [ ]:
for label, content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            df_tmp[label +"_is_missing"] = pd.isnull(content)
            df_tmp[label] = content.fillna(content.median())
        

In [ ]:
for label, content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)

In [ ]:
for label, content in df_tmp.items():
    if not pd.api.types.is_numeric_dtype(content):
        print(label)

In [ ]:
for label, content in df_tmp.items():
    if not pd.api.types.is_numeric_dtype(content):
        df_tmp[label+"_is_missing"] = pd.isnull(content)
        df_tmp[label] = pd.Categorical(content).codes + 1

In [ ]:
df_tmp.head().T

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_jobs= -1, random_state=42)

In [ ]:
model.fit(df_tmp.drop("SalePrice", axis=1), df_tmp["SalePrice"])

In [ ]:
model.score(df_tmp.drop("SalePrice", axis=1), df_tmp["SalePrice"])

In [ ]:
df_tmp["saleYear"]

In [ ]:
df_train = df_tmp[df_tmp["saleYear"] != 2012]
df_val = df_tmp[df_tmp["saleYear"] == 2012]

In [ ]:
X_train, y_train = df_train.drop("SalePrice", axis=1), df_train["SalePrice"]
X_val, y_val = df_val.drop("SalePrice", axis=1), df_val["SalePrice"]

X_train.shape, y_train.shape, X_val.shape, y_val.shape

In [ ]:
# Create evaluation function (the competition uses Root Mean Square Log Error)
from sklearn.metrics import mean_squared_log_error, mean_absolute_error

def rmsle(y_test, y_preds):
    return np.sqrt(mean_squared_log_error(y_test, y_preds))

def show_score(model):
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_val)
    scores = {"Training MAE": mean_absolute_error(y_train, train_preds),
              "Valid MAE": mean_absolute_error(y_val, val_preds),
              "Training RMSLE": rmsle(y_train, train_preds),
              "Valid RMSLE": rmsle(y_val, val_preds),
              "Training R^2": model.score(X_train, y_train),
              "Valid R^2": model.score(X_val, y_val)}
    return scores



In [49]:
model = RandomForestRegressor(n_jobs=-1, max_samples=10000)
model.fit(X_train, y_train)

RandomForestRegressor(max_samples=10000, n_jobs=-1)

In [50]:
show_score(model)

{'Training MAE': 5559.786588095979,
 'Valid MAE': 7228.527628099889,
 'Training RMSLE': 0.2574448747814875,
 'Valid RMSLE': 0.2981657638761318,
 'Training R^2': 0.8606404398262768,
 'Valid R^2': 0.8318813566707991}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

rf_grid = {"n_estimators": np.arange(10, 1000, 10),
           "max_depth": [None, 3, 5, 10],
           "min_samples_split": np.arange(2,20,2),
           "min_samples_leaf": np.arange(1,20,2),
           "max_features": [0.5, 1, "sqrt", "auto"],
           "max_samples": [10000]}

rs_model = RandomizedSearchCV(RandomForestRegressor(),
                              param_distributions = rf_grid,
                              n_iter = 20,
                              cv = 5,
                              verbose = True)
rs_model.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [ ]:
rs_model.best_params_

In [ ]:
model_score(rs_model)

In [ ]:
`

In [ ]:
def Process_data(df):
    df["saleYear"] = df.saledate.dt.year
    df["saleMonth"] = df.saledate.dt.moth
    df["saleDay"] = df.saledate.dt.day
    
    df.drop("saledate", axis=1)
    
    for label, content in df.items():
        if pd.api.types.is_numeric_dtype(content):
            if pd.isnull(content).sum():
                df[label + "is_missing"] = pd.isnull(content)
                df[label] = content.fillna(np.median(df.content))
        
        if not pd.api.types.is_numeric_dtype(content):
            df[label+"is_missing"] = df.isnull(content)
            df[label] = pd.Categorical(content).codes +1
    return df
